In [141]:
import os
import selenium.webdriver
import csv
import time
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [142]:
# URL structures for categories within ELectronics

url_sets=["https://www.walmart.com/browse/tv-video/all-tvs/3944_1060825_447913",
    "https://www.walmart.com/browse/computers/desktop-computers/3944_3951_132982",
         "https://www.walmart.com/browse/electronics/all-laptop-computers/3944_3951_1089430_132960",
         "https://www.walmart.com/browse/prepaid-phones/1105910_4527935_1072335",
         "https://www.walmart.com/browse/electronics/portable-audio/3944_96469",
         "https://www.walmart.com/browse/electronics/gps-navigation/3944_538883/",
         "https://www.walmart.com/browse/electronics/sound-bars/3944_77622_8375901_1230415_1107398",
         "https://www.walmart.com/browse/electronics/digital-slr-cameras/3944_133277_1096663",
         "https://www.walmart.com/browse/electronics/ipad-tablets/3944_1078524"]

categories=["TVs","Desktops","Laptops","Prepaid_phones","Audio","GPS","soundbars","cameras","tablets"]

# URL structures for categories within Food

url_sets=["https://www.walmart.com/browse/coffee/ground-coffee/976759_1086446_1229651",
         "https://www.walmart.com/browse/food/coffee-filters/976759_1086446_1229655",
         "https://www.walmart.com/browse/coffee/instant-coffee/976759_1086446_1229650",
         "https://www.walmart.com/browse/snacks/976759_976787",
         "https://www.walmart.com/browse/baking/baking-ingredients/976759_976780_2881195",
         "https://www.walmart.com/browse/food/condiments-sauces-spices/976759_976786_1007682",
         "https://www.walmart.com/browse/food/grains-rice/976759_976794_1001474",
         "https://www.walmart.com/browse/water/all-waters/976759_976782_1001659_2562373"]

categories=["Ground_coffee","Coffee_filters","Instant_coffee","Snacks","Baking","Condiments","Grains","Waters"]

# URL structures for categories within Toys

url_sets=["https://www.walmart.com/browse/toys/4171",
         "https://www.walmart.com/browse/lego/all-lego-sets/4171_4186_1105635_2927326",
         "https://www.walmart.com/browse/arts-crafts-for-kids/4171_645779",
         "https://www.walmart.com/browse/games-puzzles/board-games/4171_4191_133123",
         "https://www.walmart.com/browse/cars-rc-drones-trains/all-vehicles/4171_1111647_2819545"]

categories=["Dolls_Toys","Lego","ArtCraftsKids","Puzzles","Vehicles_toys"]


# URL structures for categories with Home

url_sets=["https://www.walmart.com/browse/office-furniture/desks/4044_103150_97116_91851",
         "https://www.walmart.com/browse/home/mattresses/4044_103150_102547_539386_927959",
         "https://www.walmart.com/browse/home/dining-room-sets/4044_103150_4037_3500031",
         "https://www.walmart.com/browse/home/accent-chairs/4044_103150_4038_7389568",
         "https://www.walmart.com/browse/living-room-furniture/sectional-sofas-couches/4044_103150_4038_8106416",
         "https://www.walmart.com/browse/home/microwaves/4044_90548_132950",
         "https://www.walmart.com/browse/home/cookware-sets/4044_623679_8140341_599265",
         "https://www.walmart.com/browse/home/food-storage-containers/4044_623679_1032619_5842891?",
         "https://www.walmart.com/browse/home/drawers-carts/4044_90828_925639",
         "https://www.walmart.com/browse/home/washers-dryers/4044_90548_3295848",
         "https://www.walmart.com/browse/home/curtains/4044_133012_539105_1521917"]

categories=["Furniture_desks","Furniture_mattress","Furniture_Dining","Furniture_sofas","Appliance_Microwaves",
           "Cookware","FoodStorage","Furniture_storage","Appliance_washers","Decor_Curtains"]


In [143]:
for pg in range(len(url_sets)):
    # number of pages per category
    top_n= ["1","2","3","4","5","6","7","8","9","10"]
    url_category=url_sets[pg]
    print("Category:",categories[pg])
    final_results = []
    for i_1 in range(len(top_n)):
        print("Page number within category:",i_1)
        url_cat=url_category+"?page="+top_n[i_1]
        driver= webdriver.Chrome(executable_path='C:/Drivers/chromedriver.exe')
        driver.get(url_cat)
        body_cat = driver.find_element_by_tag_name("body").get_attribute("innerHTML")
        driver.quit()
        soupBody_cat = BeautifulSoup(body_cat)
        
        
        for tmp in soupBody_cat.find_all('div', {'class':'search-result-gridview-item-wrapper'}):

            final_results.append(tmp['data-id'])
    codelist=list(set(final_results))
    print("Total number of prods:",len(codelist))
    # base URL for product page
    url1= "https://walmart.com/ip"

    # Data Headers
    WLMTData = [["Product_code","Product_name", "Product_description","Product_URL",
                 "Breadcrumb_parent","Breadcrumb_active",
                "Product_price", "Rating_Value","Rating_Count","Recommended_Prods"]]
        
    for i in range(len(codelist)):
        #creating a list without the place taken in the first loop
        print(i)
        item_wlmt=codelist[i]
        url2=url1+"/"+item_wlmt
        #print(url2)

        try:
            driver= webdriver.Chrome(executable_path='C:/Drivers/chromedriver.exe') # Chrome driver is being used.
            print ("Requesting URL: " + url2)

            driver.get(url2)   # URL requested in browser.
            print ("Webpage found ...")
            time.sleep(3)
            # Find the document body and get its inner HTML for processing in BeautifulSoup parser.
            body = driver.find_element_by_tag_name("body").get_attribute("innerHTML")
            print("Closing Chrome ...") # No more usage needed.
            driver.quit() 				# Browser Closed.

            print("Getting data from DOM ...")
            soupBody = BeautifulSoup(body) # Parse the inner HTML using BeautifulSoup
            #print(soupBody)

            h1ProductName = soupBody.find("h1", {"class": "prod-ProductTitle prod-productTitle-buyBox font-bold"})
            divProductDesc = soupBody.find("div", {"class": "about-desc about-product-description xs-margin-top"})
            liProductBreadcrumb_parent = soupBody.find("li", {"data-automation-id": "breadcrumb-item-0"})
            liProductBreadcrumb_active = soupBody.find("li", {"class": "breadcrumb active"})
            spanProductPrice = soupBody.find("span", {"class": "price-group"})
            spanProductRating = soupBody.find("span", {"itemprop": "ratingValue"})
            spanProductRating_count = soupBody.find("span", {"class": "stars-reviews-count-node"})
            
            ################# exceptions #########################
            if divProductDesc is None:
                divProductDesc="Not Available"
            else:
                divProductDesc=divProductDesc
                
            if liProductBreadcrumb_parent is None:
                liProductBreadcrumb_parent="Not Available"
            else:
                liProductBreadcrumb_parent=liProductBreadcrumb_parent
                
            if liProductBreadcrumb_active is None:
                liProductBreadcrumb_active="Not Available"
            else:
                liProductBreadcrumb_active=liProductBreadcrumb_active
                
            if spanProductPrice is None:
                spanProductPrice="NA"
            else:
                spanProductPrice=spanProductPrice

            if spanProductRating is None or spanProductRating_count is None:
                spanProductRating=0.0
                spanProductRating_count="0 ratings"
            #print(spanProductBreadcrumb_active.text)
            else:
                spanProductRating=spanProductRating.text
                spanProductRating_count=spanProductRating_count.text


            ### Recommended Products
            reco_prods=[]
            for tmp in soupBody.find_all('a', {'class':'tile-link-overlay u-focusTile'}):
                reco_prods.append(tmp['data-product-id'])

            if len(reco_prods)==0:
                reco_prods=["Not available"]
            else:
                reco_prods=reco_prods
            #print(len(reco_prods))
            WLMTData.append([codelist[i],h1ProductName.text, divProductDesc.text,url2,
                             liProductBreadcrumb_parent.text,liProductBreadcrumb_active.text,
                            spanProductPrice.text,spanProductRating,spanProductRating_count,reco_prods])




        except Exception as e:
            print (str(e))

    df=pd.DataFrame(WLMTData)
    df.columns = df.iloc[0]
    df=df.drop(df.index[0])
    print(df.head(2))

    # Output File for Products Data. This file will have the data in comma separated form.
    df.to_csv('out_'+categories[pg]+'.csv',index=False)
        

Category: TVs
Page number within category: 0
Page number within category: 1
Page number within category: 2
Page number within category: 3
Page number within category: 4
Page number within category: 5
Page number within category: 6
Page number within category: 7
Page number within category: 8
Page number within category: 9
Total number of prods: 351
0
Requesting URL: https://walmart.com/ip/2Z0G09Y8QKOL
Webpage found ...
Closing Chrome ...
Getting data from DOM ...
1
Requesting URL: https://walmart.com/ip/4WDOHDDSZVB4
Webpage found ...
Closing Chrome ...
Getting data from DOM ...
2
Requesting URL: https://walmart.com/ip/55JT1F07JDYE
Webpage found ...
Closing Chrome ...
Getting data from DOM ...
3
Requesting URL: https://walmart.com/ip/2CR95K2Y02C2
Webpage found ...
Closing Chrome ...
Getting data from DOM ...
'NoneType' object has no attribute 'text'
4
Requesting URL: https://walmart.com/ip/0VNSL5V7Z555
Webpage found ...
Closing Chrome ...
Getting data from DOM ...
'NoneType' object has 

KeyboardInterrupt: 